# NB 1 | Data Ingestion & Cleaning
---
### Project:
Amazon Reviews Analysis (DSC 400)

### Dataset: 
SAHITYA SAHU Amazon Large Dataset (~123GB)

### NB Objective:
Load and clean a more manageable subset of Amazon reviews

**Tasks:**
1. Load dataset (in sample chunks)
2. Initial EDA: summary stats, formatting, etc.
3. Data cleaning: Missing values, duplicates, data types, etc.
4. Feature engineering: numeric & text features
5. Load next chunk & repeat
6. Save and combine the cleaned datasets

## Setup
---

In [37]:
#Must
import pandas as pd
import numpy as np
import os

#text preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#sample
import json
from pathlib import Path

#EDA plots
import matplotlib.pyplot as plt
import seaborn as sns

#Set
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)

#Might need
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lynnm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lynnm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load & Clean | Chunk Set 1 (20K)
---
Key Notes:

- 20,000 reviews loaded (20000, 12)
- 19 removed during cleaning (19981, 13)
- `review_length` created

In [38]:
#Load XL dataset in chuncks
json_path = r"C:\Users\lynnm\Downloads\All_Amazon_Review.json"
chunksize = 500_000
fraction_to_sample = 0.01
sampled_chunks = []

#Read in chunks (sample a small portion first)
for i, chunk in enumerate(pd.read_json(json_path, lines=True, chunksize=chunksize)):
    print(f"Processing chunk {i+1}...")
    sample_chunk = chunk.sample(frac=fraction_to_sample, random_state=42)
    sampled_chunks.append(sample_chunk)
    
#Stop after a few chunks to get initial look
    if i >= 3:
        break

#Combine sampled chunks
df_sample = pd.concat(sampled_chunks, ignore_index=True)
print("Sampled dataset shape:", df_sample.shape)

Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Sampled dataset shape: (20000, 12)


In [39]:
#Columns and types
print(df_sample.columns)
print(df_sample.info())

#First few rows
df_sample.head()

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image', 'style'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   overall         20000 non-null  int64 
 1   verified        20000 non-null  bool  
 2   reviewTime      20000 non-null  object
 3   reviewerID      20000 non-null  object
 4   asin            20000 non-null  object
 5   reviewerName    19998 non-null  object
 6   reviewText      19988 non-null  object
 7   summary         19995 non-null  object
 8   unixReviewTime  20000 non-null  int64 
 9   vote            3128 non-null   object
 10  image           290 non-null    object
 11  style           9906 non-null   object
dtypes: bool(1), int64(2), object(9)
memory usage: 1.7+ MB
None


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,4,True,"05 9, 2017",A2VY416R3G7WD9,B0012XPRO8,Becky Ann Hathaway,i really liked this product. never really bou...,super soft,1494288000,NaN,NaN,{'Color:': ' Lavender'}
1,4,True,"06 25, 2016",A3CHVIU4GFE180,B00J9MJDFS,Anais,Have not used this item but a couple of times....,Nice product to have as it does several things...,1466812800,2,NaN,NaN
2,4,True,"10 10, 2014",AT5L0D757JA4W,B0054EU4Z8,Luxdelux,Great cap but it doesn't really stay on my hea...,Four Stars,1412899200,NaN,NaN,{'Color:': ' Black'}
3,5,True,"03 3, 2013",A30X02Q6TP9G3N,B003CU26SY,sag412,I read this would help with my thinning edges....,Great!,1362268800,NaN,NaN,{'Size:': ' One size'}
4,5,True,"03 1, 2017",A2V16V9O1JR53A,B01GOY8LBA,N P Hoca,My dad needed this bucket. He is 94 and in a...,"But he LOVED THIS, and told everyone how great...",1488326400,5,NaN,NaN


In [40]:
#Clean missing values, convert to dt, drop duplicates, etc.

def clean_reviews(df):

#Missing review text & rating
    df = df.dropna(subset=["reviewText", "overall"]).copy()

#Vote column (commas too)
    df["vote"] = (
        df["vote"]
        .astype(str)
        .str.replace(",", "")
    )
    df["vote"] = pd.to_numeric(df["vote"], errors="coerce").fillna(0)

#reviewTime to dt
    df["reviewTime"] = pd.to_datetime(df["reviewTime"], errors="coerce")

#Create: review_length
    df["review_length"] = df["reviewText"].apply(lambda x: len(str(x)))

#Drop duplicates
    df = df.drop_duplicates(subset=["reviewerID", "asin", "reviewText"])

    return df

In [41]:
#Review Cleaning on 20K Chunk
df_chunk_clean = clean_reviews(df_sample)

df_master = df_chunk_clean.copy()

#Verify
print("Before cleaning:", df_sample.shape)
print("After cleaning:", df_clean.shape)

#Show
print("After cleaning, Chunk 1 shape:", df_master.shape)
df_master.head()

Before cleaning: (20000, 12)
After cleaning: (19981, 13)
After cleaning, Chunk 1 shape: (19981, 13)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,review_length
0,4,True,2017-05-09,A2VY416R3G7WD9,B0012XPRO8,Becky Ann Hathaway,i really liked this product. never really bou...,super soft,1494288000,0.0,NaN,{'Color:': ' Lavender'},130
1,4,True,2016-06-25,A3CHVIU4GFE180,B00J9MJDFS,Anais,Have not used this item but a couple of times....,Nice product to have as it does several things...,1466812800,2.0,NaN,NaN,117
2,4,True,2014-10-10,AT5L0D757JA4W,B0054EU4Z8,Luxdelux,Great cap but it doesn't really stay on my hea...,Four Stars,1412899200,0.0,NaN,{'Color:': ' Black'},66
3,5,True,2013-03-03,A30X02Q6TP9G3N,B003CU26SY,sag412,I read this would help with my thinning edges....,Great!,1362268800,0.0,NaN,{'Size:': ' One size'},281
4,5,True,2017-03-01,A2V16V9O1JR53A,B01GOY8LBA,N P Hoca,My dad needed this bucket. He is 94 and in a...,"But he LOVED THIS, and told everyone how great...",1488326400,5.0,NaN,NaN,439


## Load & Clean | Chunk Set 2
---
Key Notes:

- 10,000 reviews loaded (10000, 12)
- 7 number of rows removed (9993, 13)
- Master dataset after Chunk 2: (29974, 13)

In [42]:
#Set new chunk load
sampled_chunks = [] 

#Be sure: skip first 3 chunks
for i, chunk in enumerate(pd.read_json(json_path, lines=True, chunksize=chunksize)):
    if i <= 3:
        continue
    print(f"Processing chunk {i+1}...")
    
#Sample of the chunk
    sample_chunk = chunk.sample(frac=fraction_to_sample, random_state=42)
    sampled_chunks.append(sample_chunk)
    
#Stop after couple new chunks
    if i >= 5:  
        break

# Combine new sampled chunks
df_sample_new = pd.concat(sampled_chunks, ignore_index=True)
print("New sampled dataset shape:", df_sample_new.shape)

#Peek into first rows
df_sample_new.head()

Processing chunk 5...
Processing chunk 6...
New sampled dataset shape: (10000, 12)


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image,vote
0,5,True,"05 17, 2016",ASU90F4FPAP2X,B0002F58J6,{'Size:': ' Strength 3.0'},elen,It is good!,Five Stars,1463443200,NaN,NaN
1,5,True,"07 12, 2015",A3U4BCU8VBEECY,B0002F4ND8,{'Style:': ' VSBPINK Pink Stick Bag'},Debi @2014 &amp; Beyond,"When the bag arrived, I was initially shocked ...",Used it for a gig this weekend and was very ha...,1436659200,NaN,NaN
2,4,True,"10 14, 2016",AC5SIBHCOUB9L,B000EEL3U8,NaN,Bob,works good and flexable,Four Stars,1476403200,NaN,NaN
3,5,True,"08 8, 2008",AV64US4GT1CUU,B000B6WXVM,NaN,Robert Perrey,Works with great precision on a variety of ins...,The Most Precise Clip-on Tuner,1218153600,NaN,2
4,1,False,"06 25, 2017",A2AZF5BT6BYAQH,B00KSIU3VU,NaN,Amazon Customer,DO NOT BUY terrible pedal it only last for a w...,One Star,1498348800,NaN,NaN


In [43]:
#Clean chunk 2
df_chunk_clean = clean_reviews(df_sample_new)

#To: master dataframe

#drop duplicates (master and new chunk)
df_master = pd.concat([df_master, df_chunk_clean], ignore_index=True)
df_master = df_master.drop_duplicates(subset=["reviewerID", "asin", "reviewText"])
df_master.reset_index(drop=True, inplace=True)

#Verify
print("After appending Chunk 2, master DF shape:", df_master.shape)
df_master.head()

After appending Chunk 2, master DF shape: (29974, 13)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,review_length
0,4,True,2017-05-09,A2VY416R3G7WD9,B0012XPRO8,Becky Ann Hathaway,i really liked this product. never really bou...,super soft,1494288000,0.0,NaN,{'Color:': ' Lavender'},130
1,4,True,2016-06-25,A3CHVIU4GFE180,B00J9MJDFS,Anais,Have not used this item but a couple of times....,Nice product to have as it does several things...,1466812800,2.0,NaN,NaN,117
2,4,True,2014-10-10,AT5L0D757JA4W,B0054EU4Z8,Luxdelux,Great cap but it doesn't really stay on my hea...,Four Stars,1412899200,0.0,NaN,{'Color:': ' Black'},66
3,5,True,2013-03-03,A30X02Q6TP9G3N,B003CU26SY,sag412,I read this would help with my thinning edges....,Great!,1362268800,0.0,NaN,{'Size:': ' One size'},281
4,5,True,2017-03-01,A2V16V9O1JR53A,B01GOY8LBA,N P Hoca,My dad needed this bucket. He is 94 and in a...,"But he LOVED THIS, and told everyone how great...",1488326400,5.0,NaN,NaN,439


## Load & Clean | Chunk Set 3
---
Key Ideas:

- 100,000 reviews loaded (total)
- 175 rows removed after cleaning (99825, 13)
- Master Dataset after Chunk 3: (129794, 13)

In [47]:
#Load & Sample Chunks 5+
sampled_chunks = [] 

#Start: chunk 5 onward
for i, chunk in enumerate(pd.read_json(json_path, lines=True, chunksize=chunksize)):
    if i <= 5:
        continue
    print(f"Processing chunk {i+1}...")
    
#Sample portion of the chunk: to each 100k faster
    sample_chunk = chunk.sample(frac=0.02, random_state=42) 
    sampled_chunks.append(sample_chunk)
    
#Stop after enough chunks
    if i >= 15:  
        break

#Combine newly sampled chunks
df_sample_new = pd.concat(sampled_chunks, ignore_index=True)
print("New sampled dataset shape:", df_sample_new.shape)

#Peek at first rows
df_sample_new.head()

Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
New sampled dataset shape: (100000, 12)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,False,"07 11, 2010",A5Z7JNWFGLXPH,B00NNZZKAO,Mrs. S. D. Bell,I have to admit that there is nothing on the m...,All round Remedy,1278806400,3,NaN,NaN
1,4,True,"02 12, 2015",A17C88HI2UHVCF,B00W6T13OU,Amazon Customer,"The variety pack was good, but the Corner Donu...",Four Stars,1423699200,NaN,NaN,NaN
2,5,False,"04 17, 2015",A2H667ZUNLNTF1,B00R1H0KQO,Wendi Barker,I like to keep some of the fiber one bars and ...,"Great for a Quick Treat or On-The-Go Snack, Ni...",1429228800,NaN,NaN,NaN
3,4,True,"04 20, 2016",A2TFQM5S1NJLZK,B00Q8JJ23S,Dina Walker,Bought for a family that needed some household...,Four Stars,1461110400,NaN,NaN,NaN
4,5,True,"02 4, 2017",A1TAJP6J1ABZLD,B01HEZC7WW,Psych RN,My dog loved it. I wanted to get away from spe...,Dog lovers meal,1486166400,NaN,NaN,NaN


In [48]:
#Section 3 sample
df_chunk_clean = clean_reviews(df_sample_new)

#Verify cleaning
print("Before cleaning:", df_sample_new.shape)
print("After cleaning:", df_chunk_clean.shape)

#Peek at first rows
df_chunk_clean.head()

Before cleaning: (100000, 12)
After cleaning: (99825, 13)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,review_length
0,5,False,2010-07-11,A5Z7JNWFGLXPH,B00NNZZKAO,Mrs. S. D. Bell,I have to admit that there is nothing on the m...,All round Remedy,1278806400,3.0,NaN,NaN,338
1,4,True,2015-02-12,A17C88HI2UHVCF,B00W6T13OU,Amazon Customer,"The variety pack was good, but the Corner Donu...",Four Stars,1423699200,0.0,NaN,NaN,81
2,5,False,2015-04-17,A2H667ZUNLNTF1,B00R1H0KQO,Wendi Barker,I like to keep some of the fiber one bars and ...,"Great for a Quick Treat or On-The-Go Snack, Ni...",1429228800,0.0,NaN,NaN,573
3,4,True,2016-04-20,A2TFQM5S1NJLZK,B00Q8JJ23S,Dina Walker,Bought for a family that needed some household...,Four Stars,1461110400,0.0,NaN,NaN,52
4,5,True,2017-02-04,A1TAJP6J1ABZLD,B01HEZC7WW,Psych RN,My dog loved it. I wanted to get away from spe...,Dog lovers meal,1486166400,0.0,NaN,NaN,159


In [49]:
#Move Section 3 to Master DF
df_master = pd.concat([df_master, df_chunk_clean], ignore_index=True)
df_master = df_master.drop_duplicates(subset=["reviewerID", "asin", "reviewText"])
df_master.reset_index(drop=True, inplace=True)

#Verify final master shape
print("After appending Section 3, master DF shape:", df_master.shape)
df_master.head()

After appending Section 3, master DF shape: (129794, 13)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style,review_length
0,4,True,2017-05-09,A2VY416R3G7WD9,B0012XPRO8,Becky Ann Hathaway,i really liked this product. never really bou...,super soft,1494288000,0.0,NaN,{'Color:': ' Lavender'},130
1,4,True,2016-06-25,A3CHVIU4GFE180,B00J9MJDFS,Anais,Have not used this item but a couple of times....,Nice product to have as it does several things...,1466812800,2.0,NaN,NaN,117
2,4,True,2014-10-10,AT5L0D757JA4W,B0054EU4Z8,Luxdelux,Great cap but it doesn't really stay on my hea...,Four Stars,1412899200,0.0,NaN,{'Color:': ' Black'},66
3,5,True,2013-03-03,A30X02Q6TP9G3N,B003CU26SY,sag412,I read this would help with my thinning edges....,Great!,1362268800,0.0,NaN,{'Size:': ' One size'},281
4,5,True,2017-03-01,A2V16V9O1JR53A,B01GOY8LBA,N P Hoca,My dad needed this bucket. He is 94 and in a...,"But he LOVED THIS, and told everyone how great...",1488326400,5.0,NaN,NaN,439


## Save | Master Cleaned Dataset
---

In [53]:
#Save as CSV 
df_master.to_csv("Amazon_Master_Cleaned.csv", index=False)